In [3]:
import folium
import json
import pandas as pd
import numpy as np

## Clase Estacion que ayuda como estructura

In [2]:
class Estacion:
    def __init__(self, nombre, linea, latitud, longitud):
        self.nombre = nombre
        self.linea = linea
        self.latitud = latitud
        self.longitud = longitud

## Función que retorna una serie pandas con la afluencia por estacion de cada linea en determinado mes y año

In [5]:
def obtenerAfluenciaMesAnioLinea(df, linea, mes='Abril', anio=2020):
    
    columns = df.columns
    columsExclude = ['fecha', 'afluencia']
    dUniqueValues = {}

    for column in columns:
        if column not in columsExclude:
            dUniqueValues[column] = df[column].unique()


    """ Analisis """

    # Filtro
    dfFiltrado = df[(df['anio'] == anio) & (df['mes'] == mes) & (df['linea'] == linea)]
    sumaAfluenciaEstacion = dfFiltrado.groupby('estacion')['afluencia'].sum()
    return sumaAfluenciaEstacion

## Crear marcador con numero de afluencia en cada estación

In [6]:
df = pd.read_csv('afluenciastc_simple_02_2024.csv', sep=',')

# Normalizar el nombre de las lineas conforme a el archivo geojson
df['linea'] = df['linea'].apply(lambda x: x.replace('Linea ', ''))

# Guardar el resultado en otro archivo CSV
df.to_csv('afluenciastc_simple_02_2024.csv', index=False)

# Carga el archivo .geojson
with open('STC_Metro_estaciones.geojson', encoding='utf-8') as f:
    data = json.load(f)

# Centrar el mapa en la ciudad de México
mapa = folium.Map(location=[19.4163340855965, -99.0747357270116], zoom_start=12, tiles='CartoDB positron')

# Crear un diccionario para almacenar las estaciones agrupadas por línea
lineas = {}
dEstaciones = {}

lineasColores = {
    "1": "#E30A45",  # Rosa mexicano
    "2": "#0000FF",  # Azul
    "3": "#808000",  # Verde olivo
    "4": "#00FFFF",  # Cian
    "5": "#FFFF00",  # Amarillo    
    "6": "#FF0000",  # Rojo 
    "7": "#FFA500",  # Naranja
    "8": "#008000",  # Verde
    "9": "#A52A2A",  # Cafe
    "A": "#800080",   # Morado
    "B": "#696969",   # Gris
    "12": "#FFD700",  # Oro
}

# Agrupar estaciones por línea
for feature in data['features']:
    nombre = feature['properties']['NOMBRE']
    linea = feature['properties']['LINEA']
    latitud, longitud = feature['geometry']['coordinates'][::-1]
    if linea not in lineas:
        lineas[linea] = []
    lineas[linea].append((latitud, longitud))
    dEstaciones[nombre] = Estacion(nombre, linea, latitud, longitud)


    folium.CircleMarker(
        location=(latitud, longitud),
        radius=5,
        color=lineasColores.get(linea, '#000000'),
        fill=True,
        fill_color=lineasColores.get(linea, '#000000'),
        fill_opacity=0.7,
        popup=nombre
    ).add_to(mapa)

# Imprimir estaciones
# for estacion in dEstaciones.values():
#     print(f"{estacion.nombre} ({estacion.linea}): {estacion.latitud}, {estacion.longitud}")

# Imprimir afluencia
# afluencia = obtenerAfluenciaMesAnioLinea(df, 'B')
# print(afluencia)

for estacion in dEstaciones.values():
    lineaAfluencia = obtenerAfluenciaMesAnioLinea(df, estacion.linea)
    if estacion.nombre in lineaAfluencia:
        afluencia = lineaAfluencia[estacion.nombre]
        folium.Marker(
            location=(estacion.latitud, estacion.longitud),
            popup=f"{estacion.nombre} ({estacion.linea}): {afluencia}",
        ).add_to(mapa)

# Agregar polilíneas para cada línea
for linea, estaciones in lineas.items():
    color = lineasColores.get(linea, '#000000') 
    folium.PolyLine(estaciones, color=color, weight=2.5, opacity=1, popup=linea).add_to(mapa)


# Mostrar el mapa
mapa
